# Merck Coding Challenge Part 1

**Created by Jonathan Olson on May 13, 2024**

Contact: jonnycoder@gmail.*com*

## Requirements
PyPlate requires Python 3.10 or greater.

 Let's first ensure that requirement is satisfied.

In [1]:
!python --version
import sys
if not sys.version_info >= (3, 10):
   print("Python version is less than 3.10")

Python 3.10.12


In [2]:
!pip install pyplate-hte
!pip3 install jinja2==3.0.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 806.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 3.1.4
    Uninstalling Jinja2-3.1.4:
      Successfully uninstalled Jinja2-3.1.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.0+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.0+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.0+cu121 requires nvidia-cuda-nvrtc-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.0+cu121 requires nvidia-cuda-runtime-cu12==12.1.105; platform_sys

# Setup Experiment

In [3]:
import numpy as np
from pyplate import Substance, Container, Plate, Recipe

# Total runs
total_runs = 12

# Substances
pd2_catalyst = Substance.solid(name="Pd(OAc)2", mol_weight=224.51)
solvent_toluene = Substance.liquid(name="toluene", mol_weight=92.14, density=0.867)
solvent_glyme = Substance.liquid(name="glyme", mol_weight=90.12, density=0.869)
solvent_tbme = Substance.liquid(name="TBME", mol_weight=88.15, density=0.740)
solvent_dichloroethane = Substance.liquid(name="dichloroethane", mol_weight=98.96, density=1.256)
ligand_xphos = Substance.solid(name="XPhos", mol_weight=476.7)
ligand_sphos = Substance.solid(name="SPhos", mol_weight=410.53)
ligand_dppf = Substance.solid(name="dppf", mol_weight=554.38)

# Generate 12 molecular weight combinations of small molecules. Use a seed for repeatable results
np.random.seed(57)
A_mol_weights_list = np.random.uniform(100, 500, total_runs)
B_mol_weights_list = np.random.uniform(100, 500, total_runs)

# Create Ai & Bi: assume unique pairs of Ai and Bi to form 12 total reactions, not all possible pairings of Ai and Bi
A_solids = [Substance.solid(name=f"A{i}", mol_weight=mw) for i, mw in enumerate(A_mol_weights_list)]
B_solids = [Substance.solid(name=f"B{i}", mol_weight=mw) for i, mw in enumerate(B_mol_weights_list)]

# Lists
solvents = [solvent_toluene, solvent_glyme, solvent_tbme, solvent_dichloroethane]
ligands = [ligand_xphos, ligand_sphos, ligand_dppf]

# Total Experiments to guide in quantity of solutions
total_experiments = total_runs * len(solvents) * len(ligands)  # 144

# 96-well
plate_row_count = 8
plate_col_count = 12
well_count = plate_row_count * plate_col_count

temperatures = [60, 80]

# Assumption: different well plate for each temperature; a plate can only accommodate one temperature for all wells
plates60C = [Plate(name="60C - 96 Wells - A", max_volume_per_well="500 uL", rows=plate_row_count, columns=plate_col_count),
             Plate(name="60C - 96 Wells - B", max_volume_per_well="500 uL", rows=plate_row_count, columns=plate_col_count),
]
plates80C = [Plate(name="80C - 96 Wells - A", max_volume_per_well="500 uL", rows=plate_row_count, columns=plate_col_count),
             Plate(name="80C - 96 Wells - B", max_volume_per_well="500 uL", rows=plate_row_count, columns=plate_col_count)
]


# Constants
B_to_A_factor = 1.1
percent_pd = 10
percent_ligand = 15

# Ratios
mmol_A_reagent = 0.1
mmol_B_reagent = mmol_A_reagent * B_to_A_factor
mmol_pd = mmol_A_reagent * (percent_pd / 100)  # Assumption: Calculate from `A` limiting reagent only
mmol_ligand = mmol_A_reagent * (percent_ligand / 100)  # Assumption: Calculate from `A` limiting reagent only

# Concentrations - PyPlate supports ratios which will look like "0.1mmol/200uL"
reaction_volume = 200  # 200uL reaction volume
reaction_measure = "uL"
concentration_A = f"{mmol_A_reagent} mmol/{reaction_volume} {reaction_measure}"
concentration_B = f"{mmol_B_reagent} mmol/{reaction_volume} {reaction_measure}"
concentration_pd = f"{mmol_pd} mmol/{reaction_volume} {reaction_measure}"
concentration_ligand = f"{mmol_ligand} mmol/{reaction_volume} {reaction_measure}"

total_quantity = total_experiments * reaction_volume


def get_row_col(index):
    """
    Return row and column values for an 8x12 matrix, representing a standard 8x12 well plate.
    For example, index 1 is (0,0), index 1 is (0,1), etc

    Args:
        index: 1-based counter ranging from 1 to 144

    Returns: Converted row and column tuple.
    """
    zero_index = index - 1
    row = zero_index // 12
    col = zero_index % 12
    return row, col


recipe = Recipe()

# Add plates to recipe
for p in plates60C:
    recipe.uses(p)
for p in plates80C:
    recipe.uses(p)

index = 1  # tracks which well we are on for given 60C plate and 80C plate
current_plate_index = 0  # 144 reactions for given temp using 2 96-well plates

# For given temperature, we have 4 solvents * 3 ligands * 12 AiBi solids = 144 total reactions.
# We will mix each reaction one at a time, and transfer it to both 60C plate and 80C plate
# There are 2 60C plates and 2 80C plates to fill in total
for i, solvent in enumerate(solvents):
    for j, ligand in enumerate(ligands):
        for k, (a, b) in enumerate(zip(A_solids, B_solids)):
            # Start at row 1 column 1 and move to row 1 column 2 until plate is full
            row, col = get_row_col(index)

            # Each well is a unique mix of AiBi, Ligand, Solvent and pd2 catalyst
            mix = Container.create_solution(name=f"{a.name}, {b.name}, {ligand.name}, {pd2_catalyst.name} in {solvent.name}",
                                            solute=[a, b, ligand, pd2_catalyst],
                                            concentration=[concentration_A, concentration_B, concentration_ligand, concentration_pd],
                                            solvent=solvent,
                                            total_quantity=f"{total_quantity} {reaction_measure}")
            recipe.uses(mix)
            current_plate60C = plates60C[current_plate_index]
            current_plate80C = plates80C[current_plate_index]
            recipe.transfer(source=mix,
                            destination=current_plate60C[row+1, col+1],
                            quantity=f"{reaction_volume} {reaction_measure}")
            recipe.transfer(source=mix,
                            destination=current_plate80C[row + 1, col + 1],
                            quantity=f"{reaction_volume} {reaction_measure}")
            index += 1

            # Logic to move to next plate after filling all 96 wells
            if (index-1) % well_count == 0:
                current_plate_index += 1
                index = 1

# Bake Recipe and Analyze Results

In [4]:
results = recipe.bake()
print(results[plates60C[0].name].wells)
print(results[plates60C[1].name].wells)
print(results[plates80C[0].name].wells)
print(results[plates80C[1].name].wells)


[[+----------------+----------+----------+------------+-----+
  |    well A,1    |  Volume  |   Mass   |   Moles    |  U  |
  +----------------+----------+----------+------------+-----+
  | Maximum Volume |  500.0   |    -     |     -      |  -  |
  |       A0       | 13.0 uL  | 13.5 mg  | 100.0 umol |  -  |
  |       B0       | 30.0 uL  | 29.5 mg  | 110.0 umol |  -  |
  |     XPhos      |  7.0 uL  |  7.2 mg  | 15.0 umol  |  -  |
  |    Pd(OAc)2    |  2.0 uL  |  2.2 mg  | 10.0 umol  |  -  |
  |    toluene     | 148.0 uL | 128.0 mg | 1.389 mmol |  -  |
  |     Total      | 200.0 uL | 180.4 mg | 1.624 mmol | 0 U |
  +----------------+----------+----------+------------+-----+
  +----------------+----------+----------+------------+-----+
  |    well A,2    |  Volume  |   Mass   |   Moles    |  U  |
  +----------------+----------+----------+------------+-----+
  | Maximum Volume |  500.0   |    -     |     -      |  -  |
  |       A1       | 19.0 uL  | 19.2 mg  | 100.0 umol |  -  |
  |     